In [3]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=4658291252adfeb7dfdfe2919a5156c048645087e2baab729ba4b8dc1c48dfe9
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [85]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, expr
from pyspark import SparkContext
from pyspark.sql.functions import broadcast
from pyspark.sql.functions import isnan, col, sum
from pyspark.sql.functions import corr
from pyspark.ml.stat import Correlation
from pyspark.ml.clustering import KMeans, BisectingKMeans
from pyspark.ml.feature import VectorAssembler


spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [38]:
#Question No.1
df = spark.read.csv("employee.csv", header=True, inferSchema=True)

rdd1 = df.filter(col("emp_salary") < 4500)
rdd2 = df.filter(col("emp_salary") >= 4500)
RDD = rdd1.join(rdd2, on = "emp_id", how = "inner")
RDD.show()


+------+--------+--------+----------+--------------+--------+--------+----------+--------------+
|emp_id|emp_name|emp_dept|emp_salary|emp_experience|emp_name|emp_dept|emp_salary|emp_experience|
+------+--------+--------+----------+--------------+--------+--------+----------+--------------+
|   109|    Iris|      HR|    4100.0|           1.5|   Henry|      IT|    6200.0|           4.5|
+------+--------+--------+----------+--------------+--------+--------+----------+--------------+



In [40]:
#Question No.2

RDD = rdd1.join(rdd2, on='emp_id', how='left_outer')
RDD.show()


+------+--------+----------+----------+--------------+--------+--------+----------+--------------+
|emp_id|emp_name|  emp_dept|emp_salary|emp_experience|emp_name|emp_dept|emp_salary|emp_experience|
+------+--------+----------+----------+--------------+--------+--------+----------+--------------+
|   104|    Dave|        HR|    4000.0|           1.5|    null|    null|      null|          null|
|   109|    Iris|        HR|    4100.0|           1.5|   Henry|      IT|    6200.0|           4.5|
|   114|    Noah|        HR|    4200.0|           1.5|    null|    null|      null|          null|
|   117|     Roy|Accountant|    4400.0|           1.5|    null|    null|      null|          null|
|   119|   Tommy|        HR|    4000.0|           1.5|    null|    null|      null|          null|
+------+--------+----------+----------+--------------+--------+--------+----------+--------------+



In [53]:
#Question No.3
RDD = rdd1.join(rdd2, on="emp_id" , how = 'inner')

total_salary = RDD.select("emp_id", (RDD.emp_id* RDD.emp_id).alias("total_salary"))
total_salary.show()


+------+------------+
|emp_id|total_salary|
+------+------------+
|   109|       11881|
+------+------------+



In [57]:
#Question No.4

rdd1 = sc.parallelize([(1, "Nimra"), (2, "Amna"), (3, "Areeb"), (4 , "Ammara")])
rdd2 = sc.parallelize([(1, 28), (2, 38), (3, 48) , (4, 58)])

df_1 = spark.createDataFrame(rdd1 , ['id' , 'name'])
df_2 = spark.createDataFrame(rdd2 , ['id' , 'age'])

broadcast_rdd1 = sc.broadcast(dict(rdd1.collect()))
RDD = df_2.join(broadcast(df_1), on="id")
RDD.show()


+---+---+------+
| id|age|  name|
+---+---+------+
|  1| 28| Nimra|
|  2| 38|  Amna|
|  3| 48| Areeb|
|  4| 58|Ammara|
+---+---+------+



In [59]:
#Question No.5
RDD = spark.sparkContext.parallelize(range(1, 11))

accumulator = spark.sparkContext.accumulator(0)
RDD.foreach(lambda x: accumulator.add(x))

print("Sum:", accumulator.value)

Sum: 55


In [61]:
#Question No.6
file = spark.read.csv('movies.csv',header=True)
result = file.filter(file.Awards == 'Yes')
result_ = result.filter(file.Genre == 'Action')
result_final = result_.select(result_.Title , result_.Year , result_.Director)
result_final.show()

+-----+----+--------+
|Title|Year|Director|
+-----+----+--------+
+-----+----+--------+



In [62]:
#Question No.7
final_result = result.select(result.Title , result.Director)
final_result.show()

+--------------------+-------------------+
|               Title|           Director|
+--------------------+-------------------+
| Fanny and Alexander|      BergmanIngmar|
|     A Man & a Woman|      LelouchClaude|
|Un Hombre y una M...|      LelouchClaude|
|   Official StoryThe|         PuenzoLuiz|
|   Wild Strawberries|      BergmanIngmar|
|Through a Glass D...|      BergmanIngmar|
|    Cries & Whispers|      BergmanIngmar|
|   Sound of MusicThe|         WiseRobert|
|     Babette's Feast|        AxelGabriel|
|Murder on the Ori...|        LumetSidney|
|     Elephant ManThe|         LynchDavid|
|E. T. The Extra-T...|    SpielbergSteven|
|              Misery|          ReinerRob|
|              Gandhi|AttenboroughRichard|
|       Autumn Sonata|      BergmanIngmar|
|            Gaslight|        CukorGeorge|
|A Woman Called Golda|         GibsonAlan|
|           Anastasia|      LitvakAnatole|
|       Cactus Flower|           SaksGene|
|             Amadeus|        FormanMilos|
+----------

In [63]:
#Question No.8
popular = file.sort(file.Popularity.desc())
popular_final = popular.filter(popular.Awards == 'No')
popular_final.select(popular_final.Title , popular_final.Popularity).show(10)

+--------------------+----------+
|               Title|Popularity|
+--------------------+----------+
|        Tango & Cash|         9|
|  Back to the Future|         9|
|Masque of the Red...|         9|
|Eighty-Four Chari...|         9|
|        Judge Priest|         9|
|           Dead-Bang|         9|
|               Glitz|         9|
|            Morituri|         9|
|Naked Gun: From t...|         9|
|        Boxing Babes|         9|
+--------------------+----------+
only showing top 10 rows



In [64]:
#Question No.9
popular = file.sort(file.Popularity.asc())
popular_final = popular.filter(popular.Year < '1980')
popular_final.select(popular_final.Title , popular_final.Popularity , popular_final.Year).show(10)

+------------------+----------+----+
|             Title|Popularity|Year|
+------------------+----------+----+
|   White Lightning|      null|1953|
|      Drop KickThe|      null|1927|
|      Desert Rider|      null|1923|
| Bank on the Stars|      null|1954|
|           Shalako|         0|1968|
|           Airport|         0|1970|
|     Anna Christie|         0|1930|
|Shout at the Devil|         0|1976|
| Tales of Tomorrow|         0|1953|
|        Indiscreet|         1|1958|
+------------------+----------+----+
only showing top 10 rows



In [66]:
#Question No.10
movies_1990 = file.filter(file.Year < 1990).orderBy("title")
movies_1990.show()

+----+------+--------------------+---------------+--------------------+-----------------+-------------------+----------+------+-------------------+
|Year|Length|               Title|          Genre|               Actor|          Actress|           Director|Popularity|Awards|              Image|
+----+------+--------------------+---------------+--------------------+-----------------+-------------------+----------+------+-------------------+
|1968|   139|2001: A Space Ody...|Science Fiction|          DulleaKeir|   TyzackMargaret|     KubrickStanley|        83|    No|   NicholasCage.png|
|1982|    92|             48 Hrs.|         Action|           NolteNick|   O'TooleAnnette|         HillWalter|        67|    No|   NicholasCage.png|
|1963|   138|               8 1/2|          Drama| MastroianniMarcello| CardinaleClaudia|    FelliniFederico|        80|   Yes|   NicholasCage.png|
|1966|    95|A Big Hand for th...|         Comedy|          FondaHenry|   WoodwardJoanne|        CookFielder|   

In [82]:
#Question No.11
new_file = spark.read.csv('wine.csv' , header = True)
print("Number of records:", new_file.count())

missing_counts = new_file.select([sum(col(c).isNull().cast("int")).alias(c) for c in new_file.columns])
missing_percentages = missing_counts.select([(col(c) / new_file.count()).alias(c) for c in new_file.columns])

#Detecting Outliers
numerical_columns = [c for c in new_file.columns if new_file.select(c).dtypes[0][1] in ('double', 'int')]
outliers = new_file 

for column in numerical_columns:
    column_name = column  
    q1, q3 = df.approxQuantile(column_name, [0.25, 0.75], 0.01)
    iqr = q3 - q1
    lower= q1 - 1.5 * iqr
    upper = q3 + 1.5 * iqr

#Correlarion
correlation_matrix = df.select(numerical_columns).toPandas().corr()
print(correlation_matrix)

Number of records: 178
Empty DataFrame
Columns: []
Index: []


In [91]:
#Question No.12
new_file = new_file.select(*(col(c).cast("double").alias(c) for c in new_file.columns))

assembler = VectorAssembler(inputCols=new_file.columns, outputCol="features")
data = assembler.transform(new_file).select("features")

kmeans = KMeans(k=3, seed=123)
model = kmeans.fit(data)

predictions = model.transform(data)

bisecting_kmeans = BisectingKMeans(k=4, seed=13)
model = bisecting_kmeans.fit(data)

predictions = model.transform(data)
predictions.show()

+--------------------+----------+
|            features|prediction|
+--------------------+----------+
|[14.23,1.71,2.43,...|         2|
|[13.2,1.78,2.14,1...|         2|
|[13.16,2.36,2.67,...|         3|
|[14.37,1.95,2.5,1...|         3|
|[13.24,2.59,2.87,...|         1|
|[14.2,1.76,2.45,1...|         3|
|[14.39,1.87,2.45,...|         3|
|[14.06,2.15,2.61,...|         3|
|[14.83,1.64,2.17,...|         2|
|[13.86,1.35,2.27,...|         2|
|[14.1,2.16,2.3,18...|         3|
|[14.12,1.48,2.32,...|         3|
|[13.75,1.73,2.41,...|         3|
|[14.75,1.73,2.39,...|         2|
|[14.38,1.87,2.38,...|         3|
|[13.63,1.81,2.7,1...|         3|
|[14.3,1.92,2.72,2...|         3|
|[13.83,1.57,2.62,...|         2|
|[14.19,1.59,2.48,...|         3|
|[13.64,3.1,2.56,1...|         1|
+--------------------+----------+
only showing top 20 rows

